## CAI - Colony Avalanche Index - Rebalancing data for March 2025

## Configuring packages

In [16]:
import pandas as pd
import decouple
import sys
sys.path.append('../')
from methodology import MethodologyProd
from pycoingecko import CoinGeckoAPI
key =  decouple.config("CG_KEY")
cg = CoinGeckoAPI(api_key=key)

## Index Variables

In [17]:
date = '2025-03-01'
min_mcap =  0.1e8
min_weight = 0.005
max_weight = 0.3
circ_supply_threshold = 0.3
liveness_threshold = 180 # In days
liquidity_consistency = 90 # In days
coingecko_category = ["avalanche-ecosystem"]

# Calculating max slippage based on liquidity threshold
liq = 2*1e6
liq_os = liq / 2 # one-side liquidity 
trade_value_tiny = 1e2
trade_value = 1e5
max_slippage = (liq_os + trade_value_tiny) / (liq_os + trade_value) - 1
# round to the nearest .01
max_slippage = round(max_slippage, 2)

# Dataframes to remove
stablecoins = pd.DataFrame(cg.get_coins_markets('usd',category='stablecoins')).set_index('id')
ctokens = pd.DataFrame(cg.get_coins_markets('usd',category='compound-tokens')).set_index('id')
atokens = pd.DataFrame(cg.get_coins_markets('usd',category='aave-tokens')).set_index('id')

methodology_class = MethodologyProd(
                                date,
                                1,
                                "avalanche",
                                "0x48f88A3fE843ccb0b5003e70B4192c1d7448bEf0",
                                "cai_benchmark_data",
                                "cai_liquidities",
                                min_mcap,
                                min_weight,
                                max_weight,
                                100000,
                                255,
                                circ_supply_threshold,
                                liveness_threshold,
                                liquidity_consistency,
                                -0.20,
                                coingecko_category)

## Token inclusion criteria

1. **Project and token characteristics**
    1. The project’s token should have been listed on CoinGecko with pricing data at least 6 months prior to the date of inclusion in the index.
    1. The project should have a token that is native to Avalanche. This excludes wrapped variants, where the underlying tokens are locked on an alt-L1.
    1. The project should be a going concern, with a dedicated team actively building, supporting and maintaining the project.
    1. No rebasing or deflationary tokens.
    1. The project must be widely considered to be building a useful protocol or product. Projects that have ponzi characteristics at the core of their offering will not be considered.
    1. Synthetic tokens which derive their value from external price feeds are not permissible.
    1. The project’s token must not have the ability to pause token transfers.
    1. The project’s protocol or product must have significant usage.
    <br>
1. **Pricing requirements**
    1. Token should have a Chainlink price feed to allow for manipulation resistant pricing.
    <br>
1. **Token Supply requirements**
    1. The project's token must have a circulating supply greater than 30% of the max supply. In cases where a token does not have a max supply, the minting mechanics would need to be assessed.
    <br>
1. **Liquidity Requirements**
    1. The token must be listed on a supported exchange. 
    1. The token must have shown consistent DeFi liquidity on Avalanche.
    1. The token must experience less than 15% price impact for a $100k trade in both directions. 
    <br>
1. **Security Requirements**
    1. The project must have been audited by smart contract security professionals with the audit report(s) publicly available. Alternatively, the protocol must have been operating long enough to create a consensus about its safety in the decentralised finance community.

## Run Methodology

In [18]:

results,liquidity = methodology_class.main(
    single_chain="avalanche",
    df_to_remove=[stablecoins,ctokens,atokens],
    ids_to_replace=[["avalanche-2"],["benqi-liquid-staked-avax"]],
    remove_category_assets=['wrapped-bitcoin',"gmx","euro-coin","bitcoin-avalanche-bridged-btc-b","benqi-liquid-staked-avax","pangolin","chainlink"],
    values_to_update=[{'id':'pangolin','category':'total_supply','value':230000000},
                      {'id':'benqi-liquid-staked-avax','category':'market_cap','value':20000000000},
                      {'id':'benqi-liquid-staked-avax','category':'symbol','value':"sAVAX"}
                      ],
    enable_liquidity_score=True,
    weight_split_data= {"asset_to_split":"benqi-liquid-staked-avax","asset_to_receive":"gogopool-ggavax","split_ratio":0.5},
    individual_asset_weight={"coq-inu":0.05},
    onchain_oracles=["coq-inu","gogopool","gogopool-ggavax",'benqi','avalaunch','joe','benqi-liquid-staked-avax','colony']

    )
liquidity

Excluding ket, pricing data available only for 43 < 180 days
Excluding wink-3, pricing data available only for 14 < 180 days
Table created successfully
Table created successfully
solv-btc on avalanche returned an invalid API response
solv-btc recorded no valid response across any platform
solv-protocol-solvbtc-bbn on avalanche returned an invalid API response
solv-protocol-solvbtc-bbn recorded no valid response across any platform
axelar on avalanche returned an invalid API response
axelar recorded no valid response across any platform
havven on avalanche returned an invalid API response
havven recorded no valid response across any platform
0x on avalanche returned an invalid API response
0x recorded no valid response across any platform
ankr on avalanche returned an invalid API response
ankr recorded no valid response across any platform
cartesi on avalanche returned an invalid API response
cartesi recorded no valid response across any platform
kyber-network-crystal on avalanche retur

,avalanche,optimal chain,best slippage,optimal slippage,best slippage chain,liquidity score
id,,,,,,
avalanche-bridged-weth-avalanche,-0.001754,avalanche,-0.001754,-0.001754,avalanche,100.000000
benqi-liquid-staked-avax,-0.002130,avalanche,-0.002130,-0.002130,avalanche,99.962706
gogopool-ggavax,-0.002656,avalanche,-0.002656,-0.002656,avalanche,99.910552
wrapped-avax,-0.004116,avalanche,-0.004116,-0.004116,avalanche,99.765763
coq-inu,-0.017692,avalanche,-0.017692,-0.017692,avalanche,98.419400
joe,-0.030711,avalanche,-0.030711,-0.030711,avalanche,97.128173
benqi,-0.128856,avalanche,-0.128856,-0.128856,avalanche,87.394747
gogopool,-0.156110,avalanche,-0.156110,-0.156110,avalanche,84.691915
colony,-0.166920,avalanche,-0.166920,-0.166920,avalanche,83.619844


## Show Results

In [19]:
results

,symbol,market_cap,weight,weight_converted,address,blockchain
joe,JOE,8.643023e+07,0.300000,77,0x6e84a6216ea6dacc71ee8e6b0a5b7322eebc0fdd,avalanche
benqi,QI,4.740916e+07,0.154104,40,0x8729438eb15e2c8b576fcc6aecda6a148776c0f5,avalanche
benqi-liquid-staked-avax,sAVAX,2.000000e+10,0.150000,39,0x2b2c81e08f1af8835a78bb2a90ae924ace0ea4be,avalanche
gogopool-ggavax,GGAVAX,2.058743e+07,0.150000,38,0xa25eaf2906fa1a3a13edac9b9657108af7b703e3,avalanche
gogopool,GGP,3.812023e+07,0.120078,30,0x69260b9483f9871ca57f81a90d91e2f96c2cd11d,avalanche
coq-inu,COQ,5.114168e+07,0.050000,12,0x420fca0121dc28039145009570975747295f2329,avalanche
colony,CLY,1.522985e+07,0.047367,12,0xec3492a2508ddf4fdc0cd76f31f340b30d1793e6,avalanche
avalaunch,Avalaunch,0.000000e+00,0.000000,0,0xd1c3f94de7e5b45fa4edbba472491a9f4b166fc4,None


## Contract Output

In [20]:
methodology_class.output_for_contract()

0x2b2c81e08f1af8835a78bb2a90ae924ace0ea4be,0x420fca0121dc28039145009570975747295f2329,0x69260b9483f9871ca57f81a90d91e2f96c2cd11d,0x6e84a6216ea6dacc71ee8e6b0a5b7322eebc0fdd,0x8729438eb15e2c8b576fcc6aecda6a148776c0f5,0xa25eaf2906fa1a3a13edac9b9657108af7b703e3,0xd1c3f94de7e5b45fa4edbba472491a9f4b166fc4,0xec3492a2508ddf4fdc0cd76f31f340b30d1793e6
39,12,30,77,40,38,0,12
